In [1]:
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [2]:
import torch.nn as nn
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=False, nonlinearity='tanh')

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # One time step
        # We need to detach the hidden state to prevent exploding/vanishing gradients
        # This is part of truncated backpropagation through time (BPTT)
        out, hn = self.rnn(x, h0.detach())

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out


In [22]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm

import xlrd
import pandas as pd

book = xlrd.open_workbook('kinetic data for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(0,7)] for r in range(0,300)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)
book = xlrd.open_workbook('prepareddatakfoldkinetic.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_data = [[sheet.cell_value(r, c)
         for c in range(0,7)] for r in range(0,15000)]

scaler = StandardScaler()
scaler.fit(data_unreplicated)
standardised_data = scaler.transform(replicated_data)
x=torch.tensor(standardised_data)

def create_inout_sequences(input_data, tw):
    training_seq = []
    label_seq = []
    L = len(input_data)
    for i in range(0,L,tw):
        train_seq = input_data[i:i+tw,0:4]
        train_label = input_data[i:i+tw,4:7]
        training_seq.append(train_seq)
        label_seq.append(train_label)
    return training_seq,label_seq

z, t = create_inout_sequences(x, 25)
x_train = torch.stack(z)
y_train = torch.stack(t)

n_splits = 6
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))


In [23]:
import torch
import copy
learning_rate=1e-3
i = n_splits-1
batches = 4


for l in range (4,24,4):
        input_dim = 4
        hidden_dim = l
        layer_dim = 1 
        output_dim = 3
        model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
        loss_fn = torch.nn.MSELoss(reduction='mean')
        model.train()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        init_state = copy.deepcopy(model.state_dict())
        init_state_opt = copy.deepcopy(optimizer.state_dict())
        av_opt_epoch=0
        av_opt_loss=0
        for i, (train_idx, valid_idx) in enumerate(splits):
            model.load_state_dict(init_state)
            optimizer.load_state_dict(init_state_opt)
            x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
            y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
            x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
            y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
            train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
            valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
            train_loader = torch.utils.data.DataLoader(train, batch_size=batches,
                                                           shuffle=False)
            valid_loader = torch.utils.data.DataLoader(valid, batch_size=batches,
                                                           shuffle=False)
            Gp=0
            epoch=0
            opt_loss=1
            while Gp<1 and epoch<200:
                epoch+=1
                model.train()
                for x_batch, y_batch in train_loader:
                    y_pred = model(x_batch)
                    loss = loss_fn(y_pred, y_batch)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                model.eval()
                avg_val_loss = 0.
                for x_batch, y_batch in valid_loader:
                    y_pred = model(x_batch).detach()
                    avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader.dataset)
                if avg_val_loss<opt_loss:
                    opt_loss = avg_val_loss  
                    opt_epoch=epoch
                Gp=(avg_val_loss/opt_loss)-1
            av_opt_epoch+=opt_epoch/n_splits
            av_opt_loss+=opt_loss/n_splits
        print(l,av_opt_epoch,av_opt_loss)


C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().det

4 266.6666666666667 0.07980261401583752
8 266.6666666666667 0.06999901506118476
12 266.6666666666667 0.06716147395471732
16 266.6666666666667 0.06534599738195539
20 266.6666666666667 0.061071202522143736
4 266.6666666666667 0.0811865392855058
8 266.6666666666667 0.06906929485189418


KeyboardInterrupt: 